<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Pipeline de Datos de Contratación Pública
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>


## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Abril 2023|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Consideraciones para la consulta en datos abiertos**|
|**Descripción y alcance**|Script para la consulta de bases y exploración de bases en el DATAVAULT.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Equipo de Datos -GAEC<td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Ricardo Suarez</td></tr> <tr><td>*Cargo:*</td> <td>Subdirector Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

## Carga de librerías y ambiente de trabajo

En esta sección se cargan las librerías necesarias para el desarrollo del script y se establecen los parámetros de trabajo.


In [1]:
import pyodbc 
from dotenv import load_dotenv
import os
load_dotenv()

print('server: ',os.getenv("server"))   
server = os.getenv("server")
username= os.getenv("username")
password= os.getenv("password")

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

ruta_DATA="../../../../DATA/"
ruta_MUESTRAS="../../../Sample_Data/Raw/MUESTRAS/ "

server:  10.240.5.39


In [2]:
### Paquetes usados para la exploración de datos

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings
warnings.filterwarnings('ignore')


<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
DF_SI=pd.read_sql_query("SELECT *  FROM [CCE_Sandbox].[SECOPI].[V_SECOPI]", cnxn)
DF_SII=pd.read_sql_query("SELECT *  FROM [CCE_Reports].[SECOPII].[HistoricoContratos]", cnxn)

## Trabajo en datalake

En esta sección se establecen los parámetros de conexión al datalake y se cargan las librerías necesarias para la conexión y consulta de datos.

In [4]:
import pandas as pd
import pyarrow.parquet as pq
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# URL del punto final del servicio Blob y token SAS
url_blob = "https://adlindicadoresemae.blob.core.windows.net/"
token_sas = "?sv=2021-10-04&ss=btqf&srt=sco&st=2023-04-26T10%3A55%3A03Z&se=2023-07-27T10%3A55%3A00Z&sp=rwdxftlacup&sig=9%2FBvXPQRTnl4CUwGzrUpMFXbpu50nNRqix5pREicSiY%3D"

# Conexión a la cuenta de almacenamiento de Azure utilizando el token SAS
connect_str = f"BlobEndpoint={url_blob};SharedAccessSignature={token_sas}"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [ ]:
# Referencia al contenedor donde se encuentran los archivos
container_name_ent = "datosemae/bronze/Entidades"
container_client_ent = blob_service_client.get_container_client(container_name_ent)

# Nombre del archivo a leer
blob_name_ent_small = "Entidades_SECOP_II.parquet"
blob_name_ent_full = "Entidades_SECOP_II_FULL.parquet"